# Chapter 3 - Linear Regression

### ISLR 3장의 Linear Regression Python 실습 

## Linear Regression (선형 회귀분석)##
- Response의 값이 숫자인 labeled 데이타를 이용하는 **Regression** 타입 **Supervised Learning** 모델
- 빨리 돌고, 오랜 시간 많이 연구되어 특성을 잘 알고, 모델의 해석이 쉬워 널리 사용

### 사용할 주요 Python 패키지
- [pandas](http://pandas.pydata.org)  : 데이터 입출력, Munging, & etc.
- [numpy](http://www.numpy.org/)  : 수식 계산 
- [matplotlib](http://matplotlib.org/)  : 시각화 
- [seaborn](https://seaborn.github.io/index.html)  : 시각화 
- **[statsmodels](http://statsmodels.sourceforge.net/)  : 통계모델**
- **[scikit-learn](http://scikit-learn.org/stable)  : 머신러닝** 

### * [Statsmodels](http://statsmodels.sourceforge.net/) 패키지의 모델을 사용해 Linear Regression을 익힌다.*  ###

Statsmodels 의 Linear Regression 모델은 ISLR 책의 R 쓰임새와 비슷하게 사용할 수 있음

- **[R 스타일 formula로 모델 만들기](http://statsmodels.sourceforge.net/stable/example_formulas.html)**

In [ ]:
# 패키지 imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import statsmodels.formula.api as smf     # R의 formula 식 유사하게 쓰임

from hblee import st,Corrplot    # hblee.py: 웹에서 훔쳤거나, 생각없이 짠 단순 맹한 클래스 & 함수  

# notebook에 직접 그래프를 plot  
%matplotlib inline

### 실행 환경
- Anaconda 4.1.1
- 추가로 seaborn : "conda install seaborn"
- 추가로 colormap & easydev : "pip install colormap easydev"

In [ ]:
np.__version__ , pd.__version__, seaborn.__version__

In [ ]:
# package_list = ['pandas', 'numpy', 'IPython', 'seaborn', 'sklearn', 'matplotlib', 'statsmodels']
# for pack in package_list:
#     statement = 'import ' + pack
#     exec(statement)
#     print ("%s : %s" % (pack, eval(pack).__version__) ) 

## Data Load

- 책에서 사용한 **Advertising** 데이터를 load 함. 
- local 머신에서 로딩할 수도, 또는 웹에서 직접 갖고 올 수도 있다. 로딩하기 전에 데이터 구조를 잘 살핍시다  

In [ ]:
# 웹에서 직접 pandas의 DataFrame으로 읽음.  첫째 column을 row index로 사용.  
advertising = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv', index_col=0)

# or, you can read data as DataFrame from local file system.  
# advertising = pd.read_csv('Data/Advertising.csv', usecols=[1,2,3,4])   
advertising.head()    # advertising.tail() 

In [ ]:
type(advertising)

In [ ]:
advertising.shape

- 200 개의 row (레코드, observation, sample)이 있음. Column은 4 개  

In [ ]:
advertising.index , advertising.columns    # row index, column names

In [ ]:
advertising.info()        

데이터에 대한 자세한 정보 제공 : 타입, shape, 각 feature/column의 속성   
- **자주 사용하기 바람**

In [ ]:
st(advertising)       # R의 str() 같이 동작하도록 만든 간단한 함수  



### 간단한 Exploratory Analysis: 모델링을 하기 전에 데이터의 특성을 살펴본다


In [ ]:
# seaborne 패키지를 이용해 feature들의 scatter plot을 본다  
seaborn.pairplot(advertising)

In [ ]:
# 'Sales'와 feature들간의 관계만을 scatterplot으로 나타내고, 
# R의 ggplot에서와 같이 regression line과 95% 신뢰대역을 나타내도록 함 ('kind='reg').
seaborn.pairplot(data=advertising, x_vars=['TV', 'Radio', 'Newspaper'], y_vars='Sales', size=6, aspect=0.8, kind='reg') 

In [ ]:
Corrplot(advertising).plot(fontsize='large')    # R style Corrplot 
plt.show()

- Sales와 TV간 ***강한 정비례*** 관계가 있다 

## 3.1 Simple Linear Regression : *feature가 1개*


$Y = \beta_0 + \beta_1X$

- $Y$ : response/output/target 
- $X$ : feature/input/predictor
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for $X$

Response(Y)로 *sales*, 1개의 feature(X)를 *TV*로 삼으면, 

$sales = \beta_0 + \beta_1TV$

- $\beta_0$ 와 $\beta_1$ 들을 **model coefficients (또는 weight)** 라 함 
- **simple linear regression의 학습** : **sales**와 **TV** 관계에 가장 맞는(RSS를 최소화하는) 선형식을 구성하는 $\beta_0$와 $\beta_1$을 데이터를 보고 학습해 추정한다

## Estimating the Coefficients of Linear Model 
***Statsmodels*** 을 사용해 **advertising** 데이터에 대한 linear regression 모델의 coefficient 추정
- ### [statsmodels version 0.5](http://statsmodels.sourceforge.net/stable/example_formulas.html) 부터 R 스타일 formula 형태 추가   

## Statsmodels의 Linear Model 사용하기
1. **모델 import** : 우리는 위에서 이미 "import statsmodels.formula.api as smf" 하여 관련 모듈(api)를 'smf' 라는 alias로 가져옴
2. **모델 instantiate** : 클래스 생성자를 이용해 모델을 만듬. 이 때 argument로 regression formula 포함
3. **학습 시킴** : instantiate된 모델 객체에게 fit() 명령을 내려 학습/훈련시키고, 학습된 모델을 반환 받음
4. **학습된 모델 활용** : 학습된 모델을 이용해 새로운 입력에 대해 예측을 하던가 등, 적절한 일거리를 줌


In [ ]:
# 1. 모델 import : 모델을 포함하는 모듈을 이미 import 했음  

# 2. Model Instantiation: Ordinary Least Squares (ols) 방식 linear regression 모델 만들기
#    - 입력 데이터는 DataFrame 타입 

lm = smf.ols(formula='Sales ~ TV', data=advertising)   

# 'advertising' DataFrame에서 'Sales' column을 resonse로, 'TV' column을 feature로 하는
#  linear regression 모델을 만들었음 

# 3. 모델에게 학습 시키고, 그 결과인 (학습된) 모델을 'lm_learned'으로 받음 
lm_learned = lm.fit()

# 학습된 모델의 coefficients
lm_learned.params

# lm.pvalues            # p values
# lm.rsquared           # R-squared statistic 

- **lm_learned._Tab_를 쳐서 'lm_learned' 객체에 어떤 method를 쓸 수 있는 지 보도록**
  

In [ ]:
# 보통은 위 2 & 3번 과정을 연결(chaining)함  
lm = smf.ols(formula='Sales ~ TV', data=advertising).fit()   
# 학습한 모델 (즉, fit model)이 만들어졌음

print ("Coeffients:\n%s \n\np-values:\n%s , \n\nr-squared: %s " % (lm.params, lm.pvalues, lm.rsquared))

### 다음 두 개의 cell은  response와 feature간의 관계를 시각화하는 또 다른 예 

In [ ]:
# Sales를 Y-축에, TV 광고비를 X-축에 놓은 scatter plot을 그리자   
plt.scatter(advertising.TV, advertising.Sales)
plt.xlabel("TV (in 1000's)")
plt.ylabel("Sales (in 1000's)")

# 위 plot에 simple regression 선을 overlay 
X = pd.DataFrame({'TV':[advertising.TV.min(), advertising.TV.max()]})
Y_pred = lm.predict(X)
plt.plot(X, Y_pred, c='red')
plt.title("Simple Linar Regression")

In [ ]:
# seaborn 패키지를 이용할 수도 
seaborn.regplot(advertising.TV, advertising.Sales, order=1, ci=None, scatter_kws={'color':'r'})
plt.xlim(-50,350)
plt.ylim(ymin=0);

In [ ]:
lm.summary()     #  모델 전체 요약. R의 summary() 함수와 비슷 

In [ ]:
# ISLR - Table 3.1
lm.summary().tables[1]

In [ ]:
st(advertising)

## 학습된 모델 활용: 예측과 관련 이슈 들 
- 위에서 만든 모델 lm은 Advertising의 TV 변수만을 feature로 사용해 만들었음
- 위의 R-squared 값 0.612 이나 Residual은 모델을 만들 때 사용한 데이터 (Training set)를 이용해 구한 Training Performance  
- 예측분석의 목표는 training set에 대해 좋은 성능을 보이는 모델을 만듬이 아니라, 처음 보게 될 (미래)의 out-of-sample 데이터에 대해 좋은 성능을 보일 것 같은 모델을 만드는 것 (즉, generalize 잘 하여 out-of-sample 성능이 좋은 모델)
- 미래의 데이터가 지금 존재하지 않는데 현재의 모델이 미래에 어떻게 동작할 지 짐작할 수 있을까? -> **모델 평가 **


#### 예측 : 만들어진 모델 (lm)을 이용해 새로운 predictor 값 (TV)을 줄 때 'Sales' 예측은? 
- 가령, TV = 100 일 때 Sales 예측

In [ ]:
# statsmodel formula 인터페이스는 입력을 pandas의 DataFrame 같은 array 형태 데이터 구조로 주어야 함 
x_new = pd.DataFrame({'TV': [100]})    # dictionary로 df를 만드는 일반 방법 
# x_new.info()
x_new.head()

### 4. 예측 : 아래에서와 같이 'predict' 메소드를 이용 
- ** predict() 의 입력이 DataFrame 같이 array 형태로 training에 사용했던 feature들을 갖고 있어야 함**

In [ ]:
lm.predict(x_new)    # 결과인 예측치를 numpy의 ndarray로 반환 

### 손으로 계산하여 확인하면; 
$$y = \beta_0 + \beta_1x$$
$$y = 7.0326 + 0.0475 \times x$$

In [ ]:
sales_manual = lm.params.Intercept + lm.params.TV * 100
print("Manual Calculation : %6f" % sales_manual)

In [ ]:
X_new = pd.DataFrame({'TV': [100, 422, 74]})   # TV가 100, 422, 또는 74일때 Sales 예측은? 
lm.predict(X_new)




# Multiple Linear Regression

**multiple linear regression**: 여러 feature들을 사용해 response 추정 

$Y = \beta_0 + \beta_1X_1 + ... + \beta_nX_n$

***Advertising***의 TV, Radio, Newspaper들을 feature로 하고, Sales를 response로 한 multiple linear regression :

$Sales = \beta_0 + \beta_1 \times TV + \beta_2 \times Radio + \beta_3 \times Newspaper$


In [ ]:
lm_mul = smf.ols(formula='Sales ~ TV + Radio + Newspaper', data=advertising).fit()
lm_mul.summary()

### 결과 해석
- F statistic의 p-value가 매우 작으므로 (1.58e-96)  모델은 유효 (최소한 1개 이상의 variable이 response와 관련)
- TV와 Radio의 p-value는 의미있음.  하지만 Newspaper의 p-value는 0.86에 달하므로 "Newspaper가 response와 관련이 없다"라는 null-hypothesis를 거부할 수 없음. 따라서 Newspaper 변수를 모델에 포함하기에는 적합하지 않음 
- **R-squared**가 0.89로 simple linear regression (0.612) 때보다 증가. 이 모델이 최소한 simple linear regression 보다 traning set의 response를 더 잘 설명(예측)한다고 생각할 수 있음. 
- 주의: 이 R-squared는 모델을 만들 때 데이터 (즉, training set에)에 대해서 구한 것이기에 실제 환경에서도 (out-of-sample) 더 좋은 특성을 보이는 지는 확신할 수 없음 
- **Cross-validation**와 같은 평가 방법을 통해 모델이 **out-of-sample**에 대해서도 **generalize** 잘 할까 짐작해 볼 수 있음 --> 나중에 

In [ ]:
lm_mul.summary().tables[1]               # Table 3.4 of ISLR 

In [ ]:
advertising.corr()     # Table 3.5 of ISLR : correlation matrix (상관 관계)

## 3.3   Other Considerations in the Regression Model

### Qualitative Predictors

In [ ]:
# Load 'credit' data from local file system 
credit = pd.read_csv('../Data/Credit.csv', usecols=list(range(1,12)))
credit.info()


- 위 feature들의 data type (dtypes)에서 float64, int64와 같이 숫자가 아닌 'object' 인 것들은 대부분 string 타입또는 다른 클래스 타입. 이것들이 category 타입 변수일 가능성 많음.
- Feature중 Gender, Student, Married, Ethnicity 변수가 qualitative(categorical) 변수
- 400개의 row/observation이 있는데, 모든 feature들이 400 개의 non-null 값을 지님. 즉, missing value가 없음

In [ ]:
credit.head(3)

In [ ]:
credit.isnull().sum()           # 다시 missing value 없음을 확인 

In [ ]:
seaborn.pairplot(credit[['Balance','Age','Cards','Education','Income','Limit','Rating']])  # ISLR - Fig 3.6
# 실행 시간이 조금 걸림.  Wait.

In [ ]:
Corrplot(credit[['Balance','Age','Cards','Education','Income','Limit','Rating']]).plot(fontsize='large')     
plt.show()

Interpreting the ***corrplot***
- 파란색(붉은색)으로 갈수록 Positive(Negative) Correlation
- 긹죽한 타원형태가 될수록 correlation이 강함   

In [ ]:
credit.Gender.unique()               # Gender 변수는 단 2개의 category를 갖음    

## 카테고리형 변수 'Gender'를 feature로 활용

In [ ]:
lm_cat = smf.ols(formula='Balance ~ Gender', data=credit).fit()   # Gender has 2 levels -> 1 dummy variable
lm_cat.summary().tables[1]          # ISLR - Table 3.7  

In [ ]:
# Regression of Balance onto Ethnicity
lm_cat_Eth = smf.ols('Balance ~ Ethnicity', credit).fit()
lm_cat_Eth.summary()            # Table 3.8 

- F-statistic p-value가 0.957에 달해 'Balance와 Ethnicity간 관련이 없다'는 null hypothesis를 거부할 수 없기에 이 데이터에 따르면 null hypothesis를 따른는 것이 좋다.  즉, 이 모델은  **꽝!**

In [ ]:
st(credit)

**변수들 중 'Ethnicity'만 제외하려면 - formula에 feature 다 나열하기 귀찮음. 뒤에... **

In [ ]:
lm_all = smf.ols('Balance ~ Income + Limit + Rating + Cards + Age + Education + Gender + Student + Married', credit).fit()
lm_all.summary()


## Removing the Additive Assumptions : 변수간 Interaction 

In [ ]:
# TV와 Radio간 interaction term을 주고 linear model을 만들면
lm_interact = smf.ols('Sales ~ TV + Radio + TV:Radio', advertising).fit()
lm_interact.summary().tables[1]             # Table 3.9

- TV와 Radio간 interaction이 유효



In [ ]:
smf.ols('Sales ~ TV*Radio', advertising).fit().summary().tables[1]      # 앞의 formula를 이렇게 표현 가능  

In [ ]:
smf.ols('Sales ~ TV + Newspaper*Radio', advertising).fit().summary()

- Newspaper와 Radio간 interaction은 유효하지 않음  


### Interaction between qualitative variable and a quantitative variable

In [ ]:
# Income(quantitative) 과 Student(qualitative with 2 levels)간 Interaction이 없다하고 모델을 학습하면;
lm_no_interact = smf.ols('Balance ~ Income  + Student', credit).fit()   
lm_no_interact.summary()

In [ ]:
# ncome(quantitative) 과 Studen(qualitative with 2 levels)간 Interaction이 있게 만들면;
lm_interact = smf.ols('Balance ~ Income*Student', credit).fit()
lm_interact.summary()

- 'Income'과 'Student' 사이의 interaction이 없다고 생각하는 것이 옳으며, 이는 R-square 값이 거의 증가하지 않은 것을 통해서도 짐작할 수 있다.


### Non-linear relationships using polynomial regressions

In [ ]:
# load 'Auto' data
auto = pd.read_csv('../Data/Auto.csv')
auto.info()
auto.head()

**(중요) horsepower 변수가 숫자이어야 함. 그런데, 위의 auto.info()로 본 horsepower 변수 타입이 'object'로 되어 있음.  즉 숫자가 아니라고 함.  auto.head()로 보니 처음에는 분명 숫자.  따라서 horsepower 변수 중간 어디 즈음 숫자가 아닌 것이 있음 **

In [ ]:
# Find out whick rows have non-numeric value on 'horsepower' column
auto_problem = auto[auto.horsepower.apply(lambda x: not(x.isnumeric()))]
auto_problem

5개의 observation 들이 'horsepower' feature에 숫자가 아님.  원본 auto.csv 를 보고 확인  
- 위의 row들을 제거할 수도 있고, 또는 파일을 읽을 때 위의 문제가 있는 row들을 제거하고 읽을 수도 있음 

In [ ]:
# Read the data again. This time skipping problematic rows 
auto = pd.read_csv('../Data/Auto.csv', na_values='?').dropna()
auto.info()
auto.iloc[28: 34, :]

- 문제있는 row들이 제거됨을 확인


### mpg를 $horsepower$ 와  $horsepower^2$ 에 대해 regression 

In [ ]:
# OLS regression of mpg onto horsepower and squared(horsepower)
lm_quadratic = smf.ols('mpg ~ horsepower + np.square(horsepower)', data=auto).fit()
lm_quadratic.summary().tables[1]             # ISLR - Table 3.10

In [ ]:
# Polynomial regression upto 3'rd degree 
lm_deg3 = smf.ols('mpg ~ horsepower + np.power(horsepower,2) +  np.power(horsepower,3)', data=auto).fit()
lm_deg3.summary()

- R의 poly()같은 함수 만드는 것은 쉬움. 함수로 만들 가치 없음 .